In [ ]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
class Value:
    def __init__(self, data, _children=(), _op='', label=''):
        self.data = data
        self.grad = 0.0
        self._backward = lambda : None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __repr__(self):
        return f"Value(data={self.data})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
        out._backward = _backward
        
        return out
    
    def __mul__(self, other):
        other = other if isinstance (other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        out._backward = _backward
        
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), " only supporting int/float powers for now"
        out = Value(self.data**other, (self, ), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        out._backward = _backward
        
        return out
    
    def __rmul__(self, other): # other * self
        return self * other

    def __truediv__(self, other):
        return self * other**-1
    
    def __neg__(self):
        return self * -1
    
    def __sub__(self, other):
        return self + (-other)
    
    def __radd__(self, other): # other + self
        return self + other
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/ (math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad        
        out._backward = _backward
        
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self, ),'exp')
        
        def _backward():
            self.grad += out.data * out.grad 
        out._backward = _backward
        
        return out

    def backward(self):
        
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)   
        build_topo(self)
        
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [ ]:
a = Value(2.0)
b = Value(4.0)
a - b

In [ ]:
from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
#       print(v, v._prev)
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s |data %.4f | grad %.4f}" % (n.label, n.data, n.grad), shape='record')
#     dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1w1 * x2w2 + b
x1w1 = x1*w1; x1w1.label='x1*w1'
x2w2 = x2*w2; x2w2.label='x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
o = n.tanh(); o.label = 'o'
o.backward()

In [ ]:
draw_dot(o)

In [ ]:
# inputs x1,x2
x1 = Value(2.0, label='x1')
x2 = Value(0.0, label='x2')
# weights w1,w2
w1 = Value(-3.0, label='w1')
w2 = Value(1.0, label='w2')
# bias of the neuron
b = Value(6.8813735870195432, label='b')
# x1w1 * x2w2 + b
x1w1 = x1*w1; x1w1.label='x1*w1'
x2w2 = x2*w2; x2w2.label='x2*w2'
x1w1x2w2 = x1w1 + x2w2; x1w1x2w2.label='x1*w1 + x2*w2'
n = x1w1x2w2 + b; n.label = 'n'
# ----
e = (2*n).exp(); e.label = 'e'
o = (e - 1) / (e + 1); o.label = 'o'
# ---
o.backward()
draw_dot(o)

In [ ]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self,x):
        act = sum((wi*xi for wi, xi in zip(self.w,x)), self.b) 
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b]

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
    
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs
    
    def parameters(self):    
        return [p for neuro in self.neurons for p in neuro.parameters()]

class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self,x):
        for layer in self.layers:
            x = layer(x)
        return x
    
    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]


In [ ]:
    
x = [2.0, 3.0, -1.0]
n = MLP(3,[4, 4, 1])
n(x)

In [ ]:
n.parameters()

In [ ]:
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],  
]

ys = [1.0, -1.0, -1.0, 1.0] # desired targets


In [ ]:
ypred = [n(x) for x in xs]
loss = sum((yout - ygt)**2 for ygt, yout, in zip(ys, ypred))
loss

In [ ]:
for k in range(50000):
    
    # forward pass
    ypred = [n(x) for x in xs]
    loss = sum((yout - ygt)**2 for ygt, yout, in zip(ys, ypred))

    # backward pass
    for p in n.parameters():
        p.grad = 0.0
    loss.backward()
    
    # update
    for p in n.parameters():
        p.data += -0.01 * p.grad
        
    print(k, loss.data)

1884 0.0016739177901027953
1885 0.0016729946766975562
1886 0.0016720725753318563
1887 0.0016711514843486073
1888 0.001670231402094317
1889 0.001669312326919103
1890 0.0016683942571766451
1891 0.0016674771912242308
1892 0.0016665611274226909
1893 0.0016656460641364338
1894 0.0016647319997333977
1895 0.001663818932585067
1896 0.0016629068610664415
1897 0.0016619957835560689
1898 0.0016610856984359754
1899 0.0016601766040917115
1900 0.0016592684989123058
1901 0.0016583613812902439
1902 0.0016574552496215551
1903 0.0016565501023056576
1904 0.001655645937745444
1905 0.001654742754347275
1906 0.0016538405505209155
1907 0.0016529393246795668
1908 0.0016520390752398555
1909 0.0016511398006218113
1910 0.001650241499248837
1911 0.0016493441695477618
1912 0.0016484478099487818
1913 0.0016475524188854446
1914 0.0016466579947946626
1915 0.0016457645361167448
1916 0.0016448720412952948
1917 0.0016439805087772627
1918 0.0016430899370129433
1919 0.0016422003244559128
1920 0.001641311669563095
1921 0.0

2345 0.0013341528473122634
2346 0.0013335650935209565
2347 0.0013329778551829157
2348 0.0013323911316224682
2349 0.0013318049221650924
2350 0.0013312192261374835
2351 0.0013306340428674802
2352 0.0013300493716840805
2353 0.0013294652119174985
2354 0.0013288815628990573
2355 0.0013282984239612972
2356 0.0013277157944378646
2357 0.0013271336736635977
2358 0.0013265520609744832
2359 0.0013259709557076611
2360 0.0013253903572013998
2361 0.0013248102647951607
2362 0.0013242306778294985
2363 0.0013236515956461423
2364 0.0013230730175879586
2365 0.001322494942998938
2366 0.0013219173712242212
2367 0.001321340301610077
2368 0.0013207637335038926
2369 0.0013201876662542003
2370 0.001319612099210651
2371 0.0013190370317240105
2372 0.0013184624631461866
2373 0.001317888392830168
2374 0.0013173148201301053
2375 0.0013167417444012296
2376 0.0013161691649998886
2377 0.0013155970812835554
2378 0.001315025492610773
2379 0.0013144543983412325
2380 0.0013138837978356982
2381 0.0013133136904560465
2382 0

2713 0.0011478647866887974
2714 0.0011474291785073015
2715 0.0011469938996642617
2716 0.0011465589497873228
2717 0.00114612432850468
2718 0.0011456900354451375
2719 0.0011452560702379872
2720 0.0011448224325131384
2721 0.0011443891219010108
2722 0.0011439561380326081
2723 0.001143523480539497
2724 0.0011430911490537428
2725 0.001142659143208021
2726 0.0011422274626355262
2727 0.0011417961069700258
2728 0.001141365075845791
2729 0.0011409343688977044
2730 0.0011405039857611309
2731 0.0011400739260720326
2732 0.0011396441894668883
2733 0.0011392147755827245
2734 0.0011387856840571141
2735 0.0011383569145281683
2736 0.001137928466634544
2737 0.0011375003400154267
2738 0.0011370725343105416
2739 0.0011366450491601741
2740 0.001136217884205116
2741 0.0011357910390867153
2742 0.001135364513446833
2743 0.001134938306927886
2744 0.0011345124191728243
2745 0.0011340868498250912
2746 0.0011336615985287175
2747 0.0011332366649282229
2748 0.0011328120486686654
2749 0.0011323877493956292
2750 0.001

3145 0.0009860640355371423
3146 0.0009857422495463102
3147 0.0009854206729063505
3148 0.0009850993054134746
3149 0.0009847781468641983
3150 0.0009844571970552777
3151 0.0009841364557837285
3152 0.0009838159228468452
3153 0.000983495598042181
3154 0.000983175481167543
3155 0.0009828555720210012
3156 0.0009825358704008942
3157 0.0009822163761058086
3158 0.0009818970889346136
3159 0.0009815780086863976
3160 0.000981259135160554
3161 0.0009809404681567107
3162 0.000980622007474759
3163 0.0009803037529148352
3164 0.000979985704277365
3165 0.0009796678613629862
3166 0.0009793502239726372
3167 0.0009790327919074937
3168 0.0009787155649689887
3169 0.0009783985429587894
3170 0.0009780817256788664
3171 0.000977765112931396
3172 0.0009774487045188301
3173 0.000977132500243896
3174 0.0009768164999095345
3175 0.0009765007033189529
3176 0.0009761851102756322
3177 0.0009758697205832767
3178 0.0009755545340458559
3179 0.0009752395504675973
3180 0.0009749247696529722
3181 0.0009746101914066944
3182 0.0

3498 0.0008841497857743146
3499 0.0008838909204552082
3500 0.0008836322063060236
3501 0.0008833736431946434
3502 0.0008831152309891292
3503 0.0008828569695576616
3504 0.0008825988587685971
3505 0.0008823408984904508
3506 0.0008820830885918647
3507 0.0008818254289416738
3508 0.0008815679194088227
3509 0.0008813105598624295
3510 0.0008810533501717591
3511 0.0008807962902062435
3512 0.0008805393798354392
3513 0.0008802826189290634
3514 0.000880026007357004
3515 0.0008797695449892699
3516 0.0008795132316960515
3517 0.0008792570673476644
3518 0.0008790010518145683
3519 0.0008787451849674058
3520 0.0008784894666769437
3521 0.0008782338968141073
3522 0.000877978475249963
3523 0.000877723201855745
3524 0.0008774680765028151
3525 0.0008772130990626858
3526 0.0008769582694070421
3527 0.0008767035874076834
3528 0.0008764490529365741
3529 0.0008761946658658348
3530 0.0008759404260677182
3531 0.0008756863334146293
3532 0.0008754323877791326
3533 0.0008751785890339094
3534 0.0008749249370518193
3535

...

48464 6.235608977037254e-05
48465 6.23548025047124e-05
48466 6.235351529225496e-05
48467 6.235222813299631e-05
48468 6.235094102693506e-05
48469 6.234965397406493e-05
48470 6.234836697438262e-05
48471 6.23470800278861e-05
48472 6.234579313457279e-05
48473 6.234450629443632e-05
48474 6.234321950747686e-05
48475 6.234193277369089e-05
48476 6.234064609307248e-05
48477 6.233935946561861e-05
48478 6.233807289133046e-05
48479 6.233678637019744e-05
48480 6.233549990222322e-05
48481 6.233421348739937e-05
48482 6.233292712572695e-05
48483 6.23316408171975e-05
48484 6.233035456181549e-05
48485 6.232906835956762e-05
48486 6.232778221045838e-05
48487 6.232649611448268e-05
48488 6.232521007163282e-05
48489 6.23239240819138e-05
48490 6.232263814531501e-05
48491 6.232135226183738e-05
48492 6.232006643147355e-05
48493 6.231878065422348e-05
48494 6.231749493008465e-05
48495 6.231620925905025e-05
48496 6.231492364111995e-05
48497 6.231363807629014e-05
48498 6.231235256455689e-05
48499 6.231106710591721e

48793 6.193543066077865e-05
48794 6.193416072214542e-05
48795 6.193289083564095e-05
48796 6.193162100126875e-05
48797 6.193035121901737e-05
48798 6.192908148888812e-05
48799 6.192781181087997e-05
48800 6.192654218498383e-05
48801 6.192527261120066e-05
48802 6.192400308952629e-05
48803 6.192273361995646e-05
48804 6.192146420248914e-05
48805 6.192019483712003e-05
48806 6.19189255238491e-05
48807 6.191765626266964e-05
48808 6.191638705357861e-05
48809 6.191511789657403e-05
48810 6.19138487916518e-05
48811 6.191257973881174e-05
48812 6.191131073804545e-05
48813 6.191004178935268e-05
48814 6.190877289272992e-05
48815 6.190750404817534e-05
48816 6.190623525568502e-05
48817 6.190496651525548e-05
48818 6.190369782688079e-05
48819 6.190242919056073e-05
48820 6.190116060629289e-05
48821 6.189989207407094e-05
48822 6.189862359389531e-05
48823 6.189735516575871e-05
48824 6.189608678966235e-05
48825 6.189481846560048e-05
48826 6.189355019356941e-05
48827 6.189228197356792e-05
48828 6.18910138055910

49141 6.149661738237696e-05
49142 6.149536539329583e-05
49143 6.149411345524591e-05
49144 6.149286156822013e-05
49145 6.149160973221831e-05
49146 6.14903579472357e-05
49147 6.1489106213271e-05
49148 6.148785453031775e-05
49149 6.148660289837902e-05
49150 6.148535131744414e-05
49151 6.148409978751404e-05
49152 6.148284830858895e-05
49153 6.148159688065845e-05
49154 6.148034550372097e-05
49155 6.147909417778178e-05
49156 6.147784290282568e-05
49157 6.147659167885866e-05
49158 6.14753405058736e-05
49159 6.147408938386546e-05
49160 6.147283831283565e-05
49161 6.14715872927801e-05
49162 6.14703363236925e-05
49163 6.146908540557294e-05
49164 6.146783453841786e-05
49165 6.146658372222282e-05
49166 6.146533295698384e-05
49167 6.146408224270286e-05
49168 6.146283157937063e-05
49169 6.146158096698772e-05
49170 6.146033040554855e-05
49171 6.145907989505403e-05
49172 6.145782943549723e-05
49173 6.145657902687551e-05
49174 6.145532866918742e-05
49175 6.145407836242938e-05
49176 6.145282810659636e-0

49504 6.104547363256941e-05
49505 6.104423996408562e-05
49506 6.104300634551552e-05
49507 6.10417727768501e-05
49508 6.104053925809651e-05
49509 6.103930578924477e-05
49510 6.103807237029542e-05
49511 6.103683900124224e-05
49512 6.103560568208477e-05
49513 6.103437241281899e-05
49514 6.103313919344213e-05
49515 6.1031906023951155e-05
49516 6.10306729043422e-05
49517 6.102943983461231e-05
49518 6.102820681476038e-05
49519 6.102697384478096e-05
49520 6.102574092467275e-05
49521 6.102450805443335e-05
49522 6.102327523405739e-05
49523 6.102204246354093e-05
49524 6.102080974288502e-05
49525 6.101957707208442e-05
49526 6.1018344451136977e-05
49527 6.101711188003818e-05
49528 6.101587935878663e-05
49529 6.101464688737561e-05
49530 6.1013414465807425e-05
49531 6.101218209407559e-05
49532 6.101094977217939e-05
49533 6.100971750011321e-05
49534 6.1008485277873186e-05
49535 6.1007253105462615e-05
49536 6.1006020982872874e-05
49537 6.1004788910101024e-05
49538 6.1003556887146354e-05
49539 6.100232

49849 6.0622799424805965e-05
49850 6.0621582798142646e-05
49851 6.062036622035873e-05
49852 6.061914969145416e-05
49853 6.0617933211424395e-05
49854 6.061671678026917e-05
49855 6.0615500397983915e-05
49856 6.061428406456561e-05
49857 6.0613067780011805e-05
49858 6.061185154431978e-05
49859 6.0610635357485594e-05
49860 6.060941921950868e-05
49861 6.060820313038106e-05
49862 6.060698709010438e-05
49863 6.060577109867646e-05
49864 6.060455515609006e-05
49865 6.0603339262343806e-05
49866 6.0602123417436346e-05
49867 6.060090762136367e-05
49868 6.059969187412351e-05
49869 6.059847617570974e-05
49870 6.059726052612487e-05
49871 6.059604492536321e-05
49872 6.059482937342002e-05
49873 6.059361387029342e-05
49874 6.05923984159818e-05
49875 6.0591183010481454e-05
49876 6.058996765379118e-05
49877 6.058875234590413e-05
49878 6.058753708682037e-05
49879 6.058632187653655e-05
49880 6.058510671505066e-05
49881 6.05838916023548e-05
49882 6.058267653845109e-05
49883 6.058146152333434e-05
49884 6.05802

In [ ]:
ypred